# Get players salary scraping [HoopsHype](https://hoopshype.com/salaries/) site

In [1]:
!pip install lxml

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [3]:
# Main salary page URL
URL="https://hoopshype.com/salaries/players/"

# Fetch url and parse it in a BeautifulSout object
request = requests.get(URL)
html = request.text
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Get page links to salary by season
links = { a.text.strip(): a['href'] for a in soup.find("div", {"class": "salaries-team-selector-top"}).find_all("a", href=True, text=True) }
dict(list(links.items())[0:3])

{'2021/22': 'https://hoopshype.com/salaries/players/',
 '2020/21': 'https://hoopshype.com/salaries/players/2020-2021/',
 '2018/19': 'https://hoopshype.com/salaries/players/2018-2019/'}

In [5]:
# Fetch salary tables for each season and store it in a pandas DataFrame
salaries = { season: pd.read_html(link)[0] for season, link in links.items() }

In [6]:
# Small clean in fetched dataframes, just to remove a unnecessary columns and set index to player name
df_salaries = [ df.drop(columns='Unnamed: 0').set_index('Player') for df in salaries.values() ]
display(df_salaries[0].head(1))
display(df_salaries[1].head(1))
df_salaries[2].head(1)

,2021/22,2022/23,2023/24,2024/25,2025/26,2026/27
Player,,,,,,
Stephen Curry,"$45,780,966","$48,070,014","$51,915,615","$55,761,217","$59,606,817",$0


,2020/21,2020/21(*)
Player,,
Stephen Curry,"$43,006,362","$43,006,362"


,2018/19,2018/19(*)
Player,,
Stephen Curry,"$37,457,154","$38,320,489"


In [8]:
# Group all salaries dataframes in just one, using player name as index
grouped_salary = df_salaries[0].join(df_salaries[1:], how='outer')
print(f'Salaries data for {len(grouped_salary)} players\n')
grouped_salary.iloc[345:348]

Salaries data for 2768 players



,2021/22,2022/23,2023/24,2024/25,2025/26,2026/27,2020/21,2020/21(*),2018/19,2018/19(*),...,1999/00,1999/00(*),1997/98,1997/98(*),1995/96,1995/96(*),1993/94,1993/94(*),1991/92,1991/92(*)
Player,,,,,,,,,,,,,,,,,,,,,
Cade Cunningham,"$10,050,120","$10,552,800","$11,055,360","$13,940,809","$18,123,052",$0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cal Bowdler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$1,025,880","$1,591,268",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Calbert Cheaney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$2,000,000","$3,102,250","$3,600,000","$5,789,577","$2,800,000","$4,733,321","$2,000,000","$3,570,595",NaN,NaN


In [9]:
# Let's get a better number format, removing the string symbols
salary = grouped_salary.replace({'\$':'', ',':''}, regex=True)

# then converting the columns to numeric type
salary = salary.apply(pd.to_numeric)

# and replacing season names in columns
# The "(*)" in the column means it is a deflacted salary in todays value
salary.columns = salary.columns.str.replace('/', '-').str.replace('(*)', '_deflacted', regex=False)

salary.iloc[345:348]

,2021-22,2022-23,2023-24,2024-25,2025-26,2026-27,2020-21,2020-21_deflacted,2018-19,2018-19_deflacted,...,1999-00,1999-00_deflacted,1997-98,1997-98_deflacted,1995-96,1995-96_deflacted,1993-94,1993-94_deflacted,1991-92,1991-92_deflacted
Player,,,,,,,,,,,,,,,,,,,,,
Cade Cunningham,10050120.0,10552800.0,11055360.0,13940809.0,18123052.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cal Bowdler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1025880.0,1591268.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Calbert Cheaney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2000000.0,3102250.0,3600000.0,5789577.0,2800000.0,4733321.0,2000000.0,3570595.0,NaN,NaN


In [10]:
salary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2768 entries, A.C. Green to Zylan Cheatham
Data columns (total 68 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   2021-22            536 non-null    float64
 1   2022-23            536 non-null    float64
 2   2023-24            536 non-null    float64
 3   2024-25            536 non-null    float64
 4   2025-26            536 non-null    float64
 5   2026-27            536 non-null    float64
 6   2020-21            578 non-null    float64
 7   2020-21_deflacted  578 non-null    float64
 8   2018-19            576 non-null    float64
 9   2018-19_deflacted  576 non-null    float64
 10  2016-17            545 non-null    float64
 11  2016-17_deflacted  545 non-null    float64
 12  2014-15            514 non-null    float64
 13  2014-15_deflacted  514 non-null    float64
 14  2012-13            494 non-null    float64
 15  2012-13_deflacted  494 non-null    float64
 16  2010-11   

In [11]:
## Now it is time to change the DataFrame format. We want to have a final DF in the form:
## /--------------|---------|---------|---------\
## |    player    | season  | nominal |  real   |
## |--------------|---------|---------|---------|
## | Player Name  | 2018-19 | 2250000 | 2500000 |
## | Player Name  | 2019-20 | 2300000 | 2530000 |
## | Player Name  | 2020-21 | 3000000 | 3010000 |
## | Other Player | 2016-17 | 1885000 | 2035000 |
## | ...          | ...     | ...     | ...     |
## \--------------|---------|---------|---------/

# At first, let's keep track of the columns with 'nominal' and 'real' salary values
nominal_cols = [i for i in salary.columns if 'deflacted' not in i]
real_cols = [i for i in salary.columns if 'deflacted' in i]

(nominal_cols[:3], real_cols[:3])

(['2021-22', '2022-23', '2023-24'],
 ['2020-21_deflacted', '2018-19_deflacted', '2016-17_deflacted'])

In [1]:
player_salaries = pd.DataFrame()
for row in salary.iterrows():
    name = row[0]
    nominal = row[1][nominal_cols].replace(0, np.nan).rename('nominal').to_frame()
    real = row[1][real_cols].rename(lambda x: x.replace('_deflacted', '')).rename('real').to_frame()
    player = nominal.join(real).sort_index().reset_index().rename(columns={'index': 'season'}).dropna(how='all', subset=['nominal', 'real'])
    player.insert(0, 'player', name)
    player_salaries = player_salaries.append(player)

NameError: name 'pd' is not defined

In [13]:
player_salaries.iloc[500:520]

,player,season,nominal,real
17,Amir Johnson,2007-08,3666666.0,4536819.0
18,Amir Johnson,2008-09,3666667.0,4319885.0
19,Amir Johnson,2009-10,3666666.0,4382411.0
20,Amir Johnson,2010-11,5000000.0,5913724.0
21,Amir Johnson,2011-12,5500000.0,6281547.0
22,Amir Johnson,2012-13,6000000.0,6740436.0
23,Amir Johnson,2013-14,6500000.0,7176238.0
24,Amir Johnson,2014-15,7000000.0,7571353.0
25,Amir Johnson,2015-16,12000000.0,12963417.0
26,Amir Johnson,2016-17,12000000.0,12835406.0


In [14]:
len(player_salaries)

15899

In [15]:
player_salaries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15899 entries, 0 to 30
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   player   15899 non-null  object 
 1   season   15899 non-null  object 
 2   nominal  15899 non-null  float64
 3   real     14550 non-null  float64
dtypes: float64(2), object(2)
memory usage: 621.1+ KB


In [16]:
player_salaries.to_csv('../data/salary.csv', index=False)